# 2022C2: Plotting SST1 RSoXS Data

## Imports

In [ ]:
## Imports
import PyHyperScattering as phs
import pathlib
import sys
import io
import ast
import json
import datetime
import dask.array as da
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from tqdm.auto import tqdm
import dask.array as da
from tiled.client import from_profile, from_uri
import subprocess


print(f'Using PyHyperScattering Version: {phs.__version__}')

## Define Paths

In [ ]:
def make_para_perp_DAs(datasets, sample_name, edge, intensity_type, pol, qlims, chi_width):
    # select dataarray to plot
    DS = datasets[f'polar_{edge}']
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    sliced_DA = DA.sel(polarization=pol, q=slice(qlims[0],qlims[1]))

    # calculate ISI dataarrays
    if pol==0:
        para_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        perp_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))
    elif pol==90:
        perp_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        para_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))   
        
    return para_DA, perp_DA

def make_para_perp_DAs_saxs(datasets, sample_name, edge, intensity_type, pol, qlims, chi_width):
    # select dataarray to plot
    DS = datasets[f'polar_{edge}']
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    sliced_DA = DA.sel(polarization=pol, q=slice(qlims[0],qlims[1]))

    # calculate ISI dataarrays
    if pol==0:
        para_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        perp_DA = sliced_DA.rsoxs.slice_chi(-90, chi_width=(chi_width/2))
    elif pol==90:
        perp_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        para_DA = sliced_DA.rsoxs.slice_chi(-90, chi_width=(chi_width/2))   
        
    return para_DA, perp_DA

In [ ]:
## Define directory paths
# Define directory paths

userPath = pathlib.Path('/nsls2/users/alevin')
# propPath = pathlib.Path('/nsls2/data/sst/proposals/2023-2/pass-311130')
# outPath = propPath.joinpath('processed_data')
# zarrsPath = outPath.joinpath('zarrs')


propPath = pathlib.Path('/nsls2/data/sst/proposals/2022-2/pass-309180')
outPath = propPath.joinpath('processed_data')
zarrsPath = outPath.joinpath('zarrs')
plotsPath = outPath.joinpath('prsoxs_plots')

## Load DataArrays 

In [ ]:
sorted([f.name for f in zarrsPath.glob('*')])

In [ ]:
# load dictionary of rsoxs datasets
rsoxs_datasets = {}
# for key in ['cartesian_carbon', 'cartesian_nitrogen', 'cartesian_oxygen', 
#             'polar_carbon', 'polar_nitrogen', 'polar_oxygen']:
for key in ['cartesian_carbon', 'polar_carbon']:
    key_start = key.split('_')[0]
    key_end = key.split('_')[1]       
    # zarrPath = list(zarrsPath.glob(f'{key_start}*{key_end}*-waxs.zarr'))[0]
    zarrPath = list(zarrsPath.glob(f'{key_start}*{key_end}*-saxs.zarr'))[0]
    rsoxs_datasets[key] = xr.open_zarr(zarrPath)
    
    # Compute any dask coordiantes
    for coord_name, coord_data in rsoxs_datasets[key].coords.items():
        if isinstance(coord_data.data, da.Array):
            rsoxs_datasets[key].coords[coord_name] = coord_data.compute()
    
    # # Add bare SiN substrate subtracted data variables if necessary
    # if key_start == 'cartesian':
    #     DS = rsoxs_datasets[key]
    #     DS['sin_sub_raw_intensity'] = DS['raw_intensity'] - DS['raw_intensity'].sel(sample_name='BareSiN')
    #     DS['sin_sub_corr_intensity'] = DS['corr_intensity'] - DS['corr_intensity'].sel(sample_name='BareSiN')
    #     rsoxs_datasets[key] = DS

In [ ]:
# DS['alo_sub_raw_intensity'] = DS['raw_intensity'] - DS['raw_intensity'].sel(sample_name='BareAlO')
# DS['alo_sub_corr_intensity'] = DS['corr_intensity'] - DS['corr_intensity'].sel(sample_name='BareAlO')

cmap = plt.cm.turbo.copy()
cmap.set_bad(cmap.get_under())

## Intensity Plots

### 1. Detector movies

In [ ]:
sample_guide = {
    'andrew1': 'PM6-Y6-CF',
    'andrew2': 'PM6-Y6-CFCN',
    'andrew3': 'PM6-Y6-Tol',
    'andrew4': 'PM6-Y7-CF',
    'andrew5': 'PM6-Y7-CFCN',
    'andrew6': 'PM6-Y7-Tol',
    'andrew7': 'PM6-Y7BO-CF',
    'andrew8': 'PM6-Y7BO-CFCN',
    'andrew9': 'PM6-Y7BO-Tol',
    'andrew10': 'PM6-Y12-CF',
    'andrew11': 'PM6-Y12-CFCN',
    'andrew12': 'PM6-Y12-Tol',
    'andrew13': 'PM7D5-Y6-CF',
    'andrew14': 'PM7D5-Y6-Tol',
    'andrew15': 'PM7D5-Y247-CF',
    'andrew16': 'PM7D5-Y247-Tol',
    'andrew17': 'PM7D5-Y12-CF',
    'andrew18': 'PM7D5-Y12-CF_2',
    'andrew19': 'PM7D5-Y12-Tol',
    'andrew20': 'PM7D5-Y12-Tol_2'
}

In [ ]:
# Select Dataset
edge = 'carbon'
DS = rsoxs_datasets[f'cartesian_{edge}']
# DS = DS.assign_coords({'sample_name': [sample_guide[sample_name] for sample_name in DS.sample_name.values]})
bcx = DS['raw_intensity'].beamcenter_x
bcy = DS['raw_intensity'].beamcenter_y


# Select Plotting Parameters
# pol = 90
energy = 285
# energy=400
pix_size = 500
pix_x_slice = slice(bcx-(pix_size/2), bcx+(pix_size/2))
pix_y_slice = slice(bcy-(pix_size/2), bcy+(pix_size/2))
cmin = 5e8
cmax = 1e11

# Select DataArray
# sample_name = 'PM6-Y6_3000_dSiN'
for pol in [0, 90]:
    for sample_name in tqdm(DS.sample_name.values, desc=f'Pol = {pol}°'):
        intensity_type = 'corr'
        DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
        # cmin = float(DA.where(sliced_DA>0).compute().quantile(0.35))
        # cmax = float(DA.where(sliced_DA>0).compute().quantile(0.9999))

        # Plot
        sliced_DA = DA.sel(polarization=pol, pix_x=pix_x_slice, pix_y=pix_y_slice).sel(energy=energy,method='nearest')
        # print(f'cmin={cmin}, cmax={cmax}')
        ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), x='qx', y='qy', cmap=cmap, norm=LogNorm(cmin,cmax))
        ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
        ax.figure.set_tight_layout(True)
        ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
        ax.colorbar.set_label('Double-Norm-Corrected Intensity [arb. units]', rotation=270, labelpad=12)
        ax.figure.savefig(plotsPath.joinpath('detector_movies_carbon_saxs', f'{sample_name}_{edge}_{intensity_type}_pol{pol}deg.jpeg'), dpi=120)
        # plt.show()
        plt.close('all')

In [ ]:
# Select Dataset
edge = 'carbon'
DS = rsoxs_datasets[f'cartesian_{edge}']

bcx = DS['raw_intensity'].beamcenter_x
bcy = DS['raw_intensity'].beamcenter_y

# Select Plotting Parameters
# pol = 90
pix_size = 500
pix_x_slice = slice(bcx-(pix_size/2), bcx+(pix_size/2))
pix_y_slice = slice(bcy-(pix_size/2), bcy+(pix_size/2))
intensity_type = 'corr'

# Select DataArray
# sample_name = 'PM6-Y6_3000_dSiN'
for pol in [0, 90]:
    for sample_name in tqdm(DS.sample_name.values, desc=f'Polarization {pol}°'):
        DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']

        output_path = plotsPath.joinpath('detector_movies_carbon_saxs', f'{sample_name}_{edge}_{intensity_type}_pol{pol}deg.mp4')

        # FFmpeg command. This is set up to accept data from the pipe and use it as input, with PNG format.
        # It will then output an H.264 encoded MP4 video.
        cmd = [
            'ffmpeg',
            '-y',  # Overwrite output file if it exists
            '-f', 'image2pipe',
            '-vcodec', 'png',
            '-r', '23',  # Frame rate
            '-i', '-',  # The input comes from a pipe
            '-vcodec', 'libx264',
            '-pix_fmt', 'yuv420p',
            '-crf', '27',  # Set the quality (lower is better, 17 is often considered visually lossless)
            str(output_path)
        ]

        # Start the subprocess
        proc = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Loop through the energy dimension and send frames to FFmpeg
        for energy in tqdm(DA.energy.values, desc=f'Making the {sample_name} movie'):
            # Make & customize plot
            sliced_DA = DA.sel(polarization=pol, pix_x=pix_x_slice, pix_y=pix_y_slice).sel(energy=energy, method='nearest')
            ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), x='qx', y='qy', cmap=cmap, norm=LogNorm(cmin,cmax))
            ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
            ax.figure.set_tight_layout(True)   
            ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
            ax.colorbar.set_label('Double-Norm-Corrected Intensity [arb. units]', rotation=270, labelpad=12)

            # Save figure to a PNG buffer
            # ax.figure.savefig(plotsPath.joinpath('detector_movies/frames', f'energy-{round(energy)}_frame-{i:03d}.png'), dpi=120)

            buf = io.BytesIO()
            ax.figure.savefig(buf, format='png')
            buf.seek(0)

            # Write the PNG buffer data to the process
            proc.stdin.write(buf.getvalue())
            plt.close('all')

        # Finish the subprocess
        out, err = proc.communicate()
        if proc.returncode != 0:
            print(f"Error: {err}")


### 2. ISI Plots

In [ ]:
# DA.sel(sample_name='PM6-Y6_3000_dSiN', polarization=0, q=slice(0.001, 0.03)).sel(energy=398.8, method='nearest').plot(norm=LogNorm(1e9, 1e11), cmap=plt.cm.turbo)
# plt.show()

In [ ]:
rsoxs_datasets['polar_carbon']

In [ ]:
perp_DA

In [ ]:
perp_DA.sel(energy=285.1,method='nearest').compute().interpolate_na(dim='chi').plot(cmap=cmap, norm=LogNorm(5e8,1e11))
# perp_DA.sel(energy=285.1,method='nearest').compute().plot(cmap=cmap, norm=LogNorm(5e8,1e11))

In [ ]:
# make selection
# sample_name = 'PM6-Y6_3000_dSiN'
edge = 'carbon'
intensity_type = 'corr'
pol = 90
qlims = (0.0015, 0.008)
chi_width = 30

energy_min = 283.5
energy_max = 289

for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'][f'{intensity_type}_intensity'].sortby('sample_name').sample_name.values):
    # para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)  
    para_DA, perp_DA = make_para_perp_DAs_saxs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)  

#     pol_paras = []
#     pol_perps = []
#     for pol in [0, 90]:
#         para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
#         pol_paras.append(para_DA)
#         pol_perps.append(perp_DA)

#     pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, 30,len(pol_paras[1].chi.values))})
#     pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, 30,len(pol_paras[1].chi.values))})
#     pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, 30,len(pol_perps[1].chi.values))})
#     pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, 30,len(pol_perps[1].chi.values))})

#     para_DA = (pol_paras[0] + pol_paras[1])/2
#     perp_DA = (pol_perps[0] + pol_perps[1])/2
    
#     para_DA = para_DA.assign_coords({'polarization':'avg'})
#     perp_DA = perp_DA.assign_coords({'polarization':'avg'})
    
#     pol = str(para_DA.polarization.values)

    # slice ISI data
    para_ISI = para_DA.compute().interpolate_na(dim='chi').mean('chi').mean('q')
    perp_ISI = perp_DA.compute().interpolate_na(dim='chi').mean('chi').mean('q')

    # plot
    fig, ax = plt.subplots()
    # fig, ax = plt.subplots(figsize=(7,3.5), dpi=120, tight_layout=True)

    para_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='para', yscale='log')
    perp_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='perp', yscale='log')
    # fig.suptitle(f'Integrated scattering intensity (ISI): {sample_name}', fontsize=14, x=0.55)
    fig.suptitle(f'Integrated scattering intensity (ISI): {sample_name}', fontsize=14)
    ax.set(title=f'Polarization = {pol}°, Chi Width = {chi_width}°, Q range = ({qlims[0]}, {qlims[1]}) ' + 'Å$^{-1}$', 
           xlabel='X-ray energy [eV]', ylabel='Intensity [arb. units]')
    ax.legend()
    plt.show()

    fig.savefig(outPath.joinpath('prsoxs_plots/isi_carbon_square', 
        f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_qrange-{qlims[0]}-{qlims[1]}_pol{pol}deg.png'), dpi=120)

    plt.close('all')

### 3. 2D Linecut Maps

In [ ]:
display(DS.sample_name.values)

In [ ]:
# make selection
# sample_name = 'PM7D5-Y6-Tol'
edge = 'carbon'
intensity_type = 'corr'
pol = 90
qlims = (0.0015, 0.008)
chi_width = 30
energy_min = 282
energy_max = 290

cmap = plt.cm.turbo.copy()

for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'][f'{intensity_type}_intensity'].sortby('sample_name').sample_name.values):
    para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)  

#     pol_paras = []
#     pol_perps = []
#     for pol in [0, 90]:
#         para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
#         pol_paras.append(para_DA)
#         pol_perps.append(perp_DA)

#     pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, 30,len(pol_paras[1].chi.values))})
#     pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, 30,len(pol_paras[1].chi.values))})
#     pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, 30,len(pol_perps[1].chi.values))})
#     pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, 30,len(pol_perps[1].chi.values))})

#     para_DA = (pol_paras[0] + pol_paras[1])/2
#     perp_DA = (pol_perps[0] + pol_perps[1])/2
    
#     para_DA = para_DA.assign_coords({'polarization':'avg'})
#     perp_DA = perp_DA.assign_coords({'polarization':'avg'})
    
#     pol = str(para_DA.polarization.values) 

    # Plot
    fig, axs = plt.subplots(1, 2, figsize=(11,5))
    cmin = 2e8
    cmax = 5e9

    para_DA.mean('chi').sel(energy=slice(energy_min,energy_max)).plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
    perp_DA.mean('chi').sel(energy=slice(energy_min,energy_max)).plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

    # Colorbar stuff
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
    cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    cbar.set_label(label='Double-Norm-Corrected Intensity [arb. units]', labelpad=12, rotation=270)

    fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
    fig.set(tight_layout=True)

    axs[0].set(title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
    axs[1].set(title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')

    plt.show()
    
    fig.savefig(plotsPath.joinpath('linecut_maps_carbon', 
            f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg_q{qlims[0]}-{qlims[1]}_energy{energy_min}-{energy_max}.png'), dpi=120)
    
    plt.close('all')

In [ ]:
a = para_EvsQ_avg.hvplot.line(groupby="energy", loglog=True)
a * perp_EvsQ_avg.hvplot.line(groupby="energy")

### 4. Meaned 1D intensity lineplots, selected energies

In [ ]:
# make selection
edge = 'carbon'
intensity_type = 'corr'
pol = 0
qlims = (0.0015, 0.008)
chi_width = 30


for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'][f'{intensity_type}_intensity'].sample_name.values):
    para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)  

#     pol_paras = []
#     pol_perps = []
#     for pol in [0, 90]:
#         para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
#         pol_paras.append(para_DA)
#         pol_perps.append(perp_DA)

#     pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, 30,len(pol_paras[1].chi.values))})
#     pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, 30,len(pol_paras[1].chi.values))})
#     pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, 30,len(pol_perps[1].chi.values))})
#     pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, 30,len(pol_perps[1].chi.values))})

#     para_DA = (pol_paras[0] + pol_paras[1])/2
#     perp_DA = (pol_perps[0] + pol_perps[1])/2
    
#     para_DA = para_DA.assign_coords({'polarization':'avg'})
#     perp_DA = perp_DA.assign_coords({'polarization':'avg'})
    
#     pol = str(para_DA.polarization.values) 

    # energies = para_DA.energy.sel(energy=slice(398.4, 401.5)).data
    # energies = para_DA.energy.sel(energy=slice(390, 404)).data
    # energies = para_DA.energy.sel(energy=[390, 398.8, 399.8, 401.1, 410], method='nearest').data
    # energies = para_DA.energy.sel(energy=[283, 284.2, 284.4, 284.8, 285.1, 285.8, 286.2, 287, 288, 300], method='nearest').data
    energies = para_DA.energy.sel(energy=[283, 284.2, 284.8, 285.1, 286.2, 287, 288, 300], method='nearest').data
    # energies = para_DA.energy.sel(energy=[283, 284.2, 284.4, 284.8, 285.1, 285.8, 286.2, 287, 288, 300], method='nearest').data


    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 0.9, len(energies)))

    # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
    # cax = ax.inset_axes([1.03, 0, 0.03, 1])
    # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    # cbar.set_label(label=f'Time [seconds]', labelpad=14)
    # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

    fig, axs = plt.subplots(ncols=2,figsize=(8,5), tight_layout=True)

    for j, energy in enumerate(energies):
        (para_DA.sel(energy=energy).compute().interpolate_na(dim='q').mean('chi')
         .plot.line(ax=axs[0], color=colors[j], yscale='log', xscale='log', label=np.round(energy,1)))
        (perp_DA.sel(energy=energy).compute().interpolate_na(dim='q').mean('chi')
         .plot.line(ax=axs[1], color=colors[j], yscale='log', xscale='log', label=np.round(energy,1)))

    fig.suptitle(f'Linecuts: {sample_name}')
    axs[0].set(title=f'Para, Polarization = {str(para_DA.polarization.values)}°, chi width = {chi_width}', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    axs[1].set(title=f'Perp, Polarization = {str(perp_DA.polarization.values)}°, chi width = {chi_width}', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')

    axs[0].legend(title='Energy [eV]')
    axs[1].legend(title='Energy [eV]')

    try: 
        fig.savefig(plotsPath.joinpath('linecut_plots_carbon', 
            f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_q{qlims[0]}-{qlims[1]}_pol{pol}deg.png'), dpi=120)

        plt.show()
    except ValueError:
        print(f'value error for log log plot for: {sample_name}')
        
    plt.close('all')

## Anisotropy Ratio Plots

### 1. 2D AR Maps

In [ ]:
rsoxs_datasets['polar_carbon']

In [ ]:
str(pol_paras[0].polarization.values)

In [ ]:
# make selection
# sample_name = 'PM6-Y6-CF'
edge = 'carbon'
intensity_type = 'corr'
pol = 0
qlims = (0.0015, 0.008)
chi_width = 30
energy_min = 282
energy_max = 290

sample_names = rsoxs_datasets['polar_carbon'].sample_name.values
for sample_name in tqdm(sample_names):
    para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)  

#     pol_paras = []
#     pol_perps = []
#     for pol in [0, 90]:
#         para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
#         pol_paras.append(para_DA)
#         pol_perps.append(perp_DA)

#     pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, 30,len(pol_paras[1].chi.values))})
#     pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, 30,len(pol_paras[1].chi.values))})
#     pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, 30,len(pol_perps[1].chi.values))})
#     pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, 30,len(pol_perps[1].chi.values))})

#     para_DA = (pol_paras[0] + pol_paras[1])/2
#     perp_DA = (pol_perps[0] + pol_perps[1])/2
    
#     para_DA = para_DA.assign_coords({'polarization':'avg'})
#     perp_DA = perp_DA.assign_coords({'polarization':'avg'})
    
#     pol = str(para_DA.polarization.values) 

    # Select AR data
    pol_choice = 0
    # ar_DA = (pol_paras[pol_choice].mean('chi') - pol_perps[pol_choice].mean('chi')) / (pol_paras[pol_choice].mean('chi') + pol_perps[pol_choice].mean('chi'))
    ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot
    # ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5), norm=plt.Normalize(-0.6, 0.6))
    ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5))

    ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
    ax.axes.set(title=f'{sample_name}, Polarization = {str(ar_DA.polarization.values)}, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]')
    # ax.axes.set(title=f'{sample_name}, Polarization = Averaged, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]')
    ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)


    ax.figure.savefig(plotsPath.joinpath('ar_maps_carbon', 
        f'vauto_{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg_q-{qlims[0]}-{qlims[1]}_energy{energy_min}-{energy_max}.png'), 
                      dpi=120)

    plt.show()
    plt.close('all')

In [ ]:
pol_paras[1].assign_coords({'chi': np.linspace(0, 30,len(pol_paras[1].chi.values))})

In [ ]:
pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, 30,len(pol_paras[1].chi.values))})
pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, 30,len(pol_paras[1].chi.values))})

avg_para_DA = (pol_paras[0] + pol_paras[1])/2
avg_para_DA

In [ ]:
ARvsQ_avg.hvplot.line(groupby="energy")

### 2. 1D AR Maps

## Loop for saving many plots

In [ ]:
rsoxs_datasets['polar_oxygen']

In [ ]:
# make selection
edge = 'oxygen'
intensity_type = 'corr'
qlims = (0.01, 0.08)
chi_width = 30
energy_min = 525
energy_max = 545
cmin = 5e8
cmax = 5e10

for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'].sample_name.data):
    for pol in [0, 90]:
        ### Select para & perp DataArrays
        para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)  
        
        ### ISI:
        # Slice ISI data
        para_ISI = para_DA.interpolate_na(dim='q').mean('chi').sum('q')
        perp_ISI = perp_DA.interpolate_na(dim='q').mean('chi').sum('q')
        
        # Plot
        fig, ax = plt.subplots()
        para_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='para', yscale='log')
        perp_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='perp', yscale='log')
        # para_ISI.plot.line(ax=ax, label='para', yscale='log')
        # perp_ISI.plot.line(ax=ax, label='perp', yscale='log')        
        fig.suptitle('Integrated Scattering Intensity (ISI)', fontsize=14)
        ax.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', xlabel='Photon Energy [eV]', ylabel='Double-Norm-Corrected Intensity [arb. units]')
        ax.legend()
        fig.savefig(plotsPath.joinpath('isi_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')
        
        ### Linecut Maps:
        fig, axs = plt.subplots(1, 2, figsize=(11,5))

        para_DA.mean('chi').sel(energy=slice(energy_min,energy_max)).plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
        perp_DA.mean('chi').sel(energy=slice(energy_min,energy_max)).plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

        sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
        cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
        cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
        cbar.set_label(label='Intensity [arb. units]', labelpad=12)
        fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
        fig.set(tight_layout=True)
        axs[0].set(title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
        axs[1].set(title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')
        fig.savefig(plotsPath.joinpath('linecut_maps_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')

        ### AR Maps:
        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        if para_DA.sample_name=='BareSiN':
            bareSiN_ar_DA = ar_DA.copy()
            
        # # Subtract AR from SiN:
        # ar_DA = ar_DA - bareSiN_ar_DA

        # Plot
        ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5), norm=plt.Normalize(-0.6, 0.6))
        ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
        ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]')
        ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)
        ax.figure.savefig(plotsPath.joinpath('ar_maps_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')

## Incorporating simulation output
old and incomplete

In [ ]:
import h5py

In [ ]:
whos PosixPath

In [ ]:
simsPath = rootPath.joinpath('imgs_analysis/sim_runs')

In [ ]:
[f.name for f in simPath.iterdir()]

In [ ]:
simPath = simsPath.joinpath('D1.0_a1.0_eps1.0_250pix_375size_0.5m_600steps_1')
h5path = simPath.joinpath('HDF5')
h5list = sorted(h5path.glob('E*h5'))

In [ ]:
def print_key(f, key):
    try:
        keys2 = f[key].keys()
        for key2 in keys2:
            new_key = key + '/' + key2
            print_key(f, new_key)
    except AttributeError:
        print(key)

with h5py.File(h5list[0],'r') as f:
    for key in f.keys():
        print_key(f, key)

#### The scattering pattern is contained within 'K0/projection.' If you simulated multiple incident angles there would be additional 'K#' keys. KIDList contains the vector values for each of the incident angles (propagation direction).

#### Instantiate the PyHyperScattering loader and integrator

In [ ]:
load = phs.load.cyrsoxsLoader(use_chunked_loading=True)
integ = phs.integrate.WPIntegrator(force_np_backend=True) # avoiding gpu backend for this tutorial

In [ ]:
raw = load.loadDirectory(simPath)

In [ ]:
raw

In [ ]:
remeshed = integ.integrateImageStack(raw)

In [ ]:
remeshed

In [ ]:
remeshed.sel(energy=285.1).plot()

In [ ]:
# c = cm.jet(np.linspace(0,1,len(remeshed)))

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,3),dpi=140,constrained_layout=True)
raw.sel(energy=275).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[0],add_colorbar=False)
raw.sel(energy=285).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[1],add_colorbar=False)
raw.sel(energy=295).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[2])

[{axes.set_xlim(-0.1,0.1),axes.set_ylim(-0.1,0.1)} for axes in ax]
plt.show()

In [ ]:
loaded_rsoxs.build_tiff_stack??